In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.precision", 3)
pd.options.display.float_format = '{:.2f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from IPython.core.interactiveshell import InteractiveShell
import datetime as dt
from matplotlib.lines import Line2D     

InteractiveShell.ast_node_interactivity = "all"

# I like the Unix shortcuts...
from os.path import expanduser
home = expanduser("~")

# Let's also make the figures look a bit nicer
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.family'] = 'DeJavu serif'
plt.rcParams['font.serif'] = ['Computer Modern'] 
plt.rcParams['mathtext.fontset']= 'cm'
plt.close('all')

In [2]:
# Let's start by listing our countries of interest...
countries = ("New Zealand", "United Kingdom", "United States", "Australia",
             "Chile", "Colombia", "Peru", "Taiwan", "Vietnam")
countries

('New Zealand',
 'United Kingdom',
 'United States',
 'Australia',
 'Chile',
 'Colombia',
 'Peru',
 'Taiwan',
 'Vietnam')

In [17]:
# Now, let's pull the mortality data. "pd.read_csv" pulls comma-separated values, 
# a common type of dataset that's fairly universal. df.head() prints a sample for inspection.
url3 = "https://raw.githubusercontent.com/akarlinsky/world_mortality/main/world_mortality.csv"
mortality = pd.read_csv(url3)
print(mortality.head().to_markdown())

|    | iso3c   | country_name   |   year |   time | time_unit   |   deaths |
|---:|:--------|:---------------|-------:|-------:|:------------|---------:|
|  0 | ALB     | Albania        |   2015 |      1 | monthly     |     2490 |
|  1 | ALB     | Albania        |   2015 |      2 | monthly     |     2139 |
|  2 | ALB     | Albania        |   2015 |      3 | monthly     |     2051 |
|  3 | ALB     | Albania        |   2015 |      4 | monthly     |     1906 |
|  4 | ALB     | Albania        |   2015 |      5 | monthly     |     1709 |


In [4]:
# Let's tidy our data. We first use df.loc[<some condition>] to select only a range of data that meets
# some conditions -- here, that the country is in our list. 
mortality = mortality.loc[mortality["country_name"].isin(countries)]

# Now we rename our columns to be harmonious with our later data-set, then drop the unneeded columns.
mortality["country"] = mortality["country_name"]
mortality = mortality.drop(columns=["country_name"])

mortality["week"] = mortality["time"]
mortality = mortality.drop(columns=["time", "time_unit", "iso3c"])

In [5]:
# And inspect.
mortality.head()
mortality.dtypes
mortality["deaths"].groupby(mortality["country"]).describe()

,year,deaths,country,week
640,2015,2925.00,Australia,1
641,2015,2772.00,Australia,2
642,2015,2771.00,Australia,3
643,2015,2768.00,Australia,4
644,2015,2680.00,Australia,5


year         int64
deaths     float64
country     object
week         int64
dtype: object

,count,mean,std,min,25%,50%,75%,max
country,,,,,,,,
Australia,483.00,3226.76,304.19,2678.00,2975.00,3206.00,3421.00,4217.00
Chile,442.00,2279.04,367.68,1535.00,1977.25,2256.50,2467.75,3983.00
Colombia,431.00,5014.51,1209.87,3945.00,4288.00,4580.00,5180.00,10606.00
New Zealand,494.00,657.02,70.43,511.00,606.50,646.00,701.00,968.00
Peru,301.00,3702.48,1538.32,2404.90,2915.60,3120.60,3514.80,9193.00
Taiwan,114.00,15162.76,1792.69,12122.00,13868.50,14655.00,16395.00,21033.00
United Kingdom,495.00,12188.62,2067.79,7893.00,10841.00,11806.00,13058.50,24691.00
United States,450.00,57955.90,7549.79,48285.00,52175.25,56820.00,60243.25,87415.00


In [6]:
# Let's get the data into daily format format as well. 
# We'll assume that all our data are recorded on Mondays of that week. 

# A trick for doing this: we don't actually want to add our numeric values 
# for year and week, so we'll convert them to strings (that we can then drop)
# and then add them, which just "mashes" them together. Then we can just tell
# Pandas that we have the format YEARWEEKdayofweek with %Y%W%w. 
mortality["temp1"] = mortality["year"].astype(str)
mortality["temp2"] = mortality["week"].astype(str)
mortality["datestring"] = mortality["temp1"] + mortality["temp2"]
mortality["datestring"] = mortality["datestring"] + "1" 
    # these are monthly data we'll merge with daily; no harm in just assigning
    # to first of the month
mortality["date"] = pd.to_datetime(mortality["datestring"], format="%Y%W%w")
mortality = mortality.drop(columns=["temp1", "temp2", "datestring"])

In [7]:
# And inspect.
mortality.head()
mortality.dtypes
mortality["deaths"].groupby(mortality["country"]).describe()

,year,deaths,country,week,date
640,2015,2925.00,Australia,1,2015-01-05
641,2015,2772.00,Australia,2,2015-01-12
642,2015,2771.00,Australia,3,2015-01-19
643,2015,2768.00,Australia,4,2015-01-26
644,2015,2680.00,Australia,5,2015-02-02


year                int64
deaths            float64
country            object
week                int64
date       datetime64[ns]
dtype: object

,count,mean,std,min,25%,50%,75%,max
country,,,,,,,,
Australia,483.00,3226.76,304.19,2678.00,2975.00,3206.00,3421.00,4217.00
Chile,442.00,2279.04,367.68,1535.00,1977.25,2256.50,2467.75,3983.00
Colombia,431.00,5014.51,1209.87,3945.00,4288.00,4580.00,5180.00,10606.00
New Zealand,494.00,657.02,70.43,511.00,606.50,646.00,701.00,968.00
Peru,301.00,3702.48,1538.32,2404.90,2915.60,3120.60,3514.80,9193.00
Taiwan,114.00,15162.76,1792.69,12122.00,13868.50,14655.00,16395.00,21033.00
United Kingdom,495.00,12188.62,2067.79,7893.00,10841.00,11806.00,13058.50,24691.00
United States,450.00,57955.90,7549.79,48285.00,52175.25,56820.00,60243.25,87415.00


In [8]:
# Now we do the same for the Google and Apple mobility data
url0 = ("https://raw.githubusercontent.com/ActiveConclusion/"
        "COVID19_mobility/master/summary_reports/summary_report_countries.csv")
googapp = pd.read_csv(url0)
googapp = googapp.loc[googapp['country'].isin(countries)]

In [9]:
# Let's declare these to be time-series data, too, and extract a year. 
googapp["date"] = pd.to_datetime(googapp["date"])
googapp["year"] = pd.to_datetime(googapp["date"]).dt.isocalendar().year
googapp["week"] = pd.to_datetime(googapp["date"]).dt.isocalendar().week
googapp.head()
googapp.dtypes

,country,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking,year,week
5384,Australia,2020-01-13,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,2020,3
5385,Australia,2020-01-14,NaN,NaN,NaN,NaN,NaN,NaN,2.98,1.78,1.31,2020,3
5386,Australia,2020-01-15,NaN,NaN,NaN,NaN,NaN,NaN,4.21,0.64,1.82,2020,3
5387,Australia,2020-01-16,NaN,NaN,NaN,NaN,NaN,NaN,8.63,-0.42,4.52,2020,3
5388,Australia,2020-01-17,NaN,NaN,NaN,NaN,NaN,NaN,9.08,-1.66,13.73,2020,3


country                          object
date                     datetime64[ns]
retail and recreation           float64
grocery and pharmacy            float64
parks                           float64
transit stations                float64
workplaces                      float64
residential                     float64
driving                         float64
transit                         float64
walking                         float64
year                             UInt32
week                             UInt32
dtype: object

In [10]:
# And, finally, let's collapse by week. This is probably more rigorous
# than "upcasting" our weekly data. 
googapp = googapp.groupby(["country", "year", "week"]).mean()

/var/folders/r1/v24t28293vb4sxbzx6539ddw0000gn/T/ipykernel_1846/4080963557.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  googapp = googapp.groupby(["country", "year", "week"]).mean()


In [11]:
# Now we can merge on the unique pairings of country and date
# "how = 'outer'" keeps unmerged rows, which we want since we 
# know that these DFs are of different dimensions
df = pd.merge(googapp, mortality, how="outer", on=["country", "year", "week"])

In [12]:
# Let's now make sure that our data-set is sorted and take a peek with a sorted sample. 
# Note that we have to re-set the index manually in Pandas.
df = df.sort_values(by =["country", "date"])
df = df.reset_index(drop=True)
df.head(5)
df.dtypes

# This is good -- we should expect only excess mortality data from those earlier years
# We do want the older data to construct excess mortality estimates
# but we don't need it past the time that mobility data stopped being constructed. 
df = df.loc[df["date"]<"2022-05-01"]

,country,year,week,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking,deaths,date
0,Australia,2015,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2925.00,2015-01-05
1,Australia,2015,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2772.00,2015-01-12
2,Australia,2015,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2771.00,2015-01-19
3,Australia,2015,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2768.00,2015-01-26
4,Australia,2015,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2680.00,2015-02-02


country                          object
year                             UInt32
week                             UInt32
retail and recreation           float64
grocery and pharmacy            float64
parks                           float64
transit stations                float64
workplaces                      float64
residential                     float64
driving                         float64
transit                         float64
walking                         float64
deaths                          float64
date                     datetime64[ns]
dtype: object

In [13]:
# We can now do a bit of grouping to print our results in a convenient way 
# and get a sense for what the data look like. The COVID-policy outcomes 
# should exist for only three years; deaths should go back further.

# We just keep a few columns to make this easy to assess, transposing for 
# easier accessibility. 

df.groupby(["country", "year"])[["transit", "deaths"]].apply("mean").T


country Australia                                                          \
year         2015    2016    2017    2018    2019    2020    2021    2022   
transit       NaN     NaN     NaN     NaN     NaN  -43.72  -31.98   -2.32   
deaths    3017.32 3047.23 3152.56 3059.46 3161.46 3109.49 3304.21 3497.71   

country   Chile          ... United Kingdom          United States           \
year       2016    2017  ...           2021     2022          2015     2016   
transit     NaN     NaN  ...           7.62    54.56           NaN      NaN   
deaths  1987.98 2039.33  ...       12801.21 13022.88      52057.25 52573.12   

country                                                        
year        2017     2018     2019     2020     2021     2022  
transit      NaN      NaN      NaN   -45.54   -13.38    10.36  
deaths  54057.46 54597.58 54858.33 64921.21 66490.87 68586.76  

[2 rows x 61 columns]

In [14]:
# The next step is to make some weekly excess death figures.
# We first make a df for pre-2020 data so that we have a baseline unaffected by COVID. 
pre20data = df[df["year"]<2020]

gb = pre20data.groupby(["country", "week"])["deaths"]
# I'm commenting out this next line, but you should run it if you want to see how gb works
#for i in gb:
    #print(i)
# This is quite helpful. https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_stata.html
# as is this: # https://stackoverflow.com/questions/45255236/how-is-pandas-groupby-method-actually-working
# groupby basically reframes our df as a Kronecker "merge": df = [groups]'[I_groups] +merge [data matrix]
# it then splits this into a bunch of paired objects: a vector defining unique groups and then a vector
# of all observations per group

In [15]:
df["week_mean_death"] = gb.transform("mean")
# Let's check our work quickly around the time of the transition, noting that we are now missing
# the baseline for weekly mean deaths.
df[df["date"]>"2019-11-30"].head(20)

,country,year,week,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking,deaths,date,week_mean_death
256,Australia,2019,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2988.00,2019-12-02,2918.00
257,Australia,2019,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2958.00,2019-12-09,2882.40
258,Australia,2019,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2995.00,2019-12-16,2951.40
259,Australia,2019,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2950.00,2019-12-23,2908.80
260,Australia,2019,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2881.00,2019-12-30,2850.20
261,Australia,2020,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2928.00,2020-01-06,NaN
262,Australia,2020,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2903.00,2020-01-13,NaN
263,Australia,2020,3,NaN,NaN,NaN,NaN,NaN,NaN,1.89,-1.83,2.88,2915.00,2020-01-20,NaN
264,Australia,2020,4,NaN,NaN,NaN,NaN,NaN,NaN,2.40,0.39,5.27,3010.00,2020-01-27,NaN
265,Australia,2020,5,NaN,NaN,NaN,NaN,NaN,NaN,-2.70,-0.64,-0.62,2911.00,2020-02-03,NaN


In [16]:
# The solution here is to forward fill once we've grouped the data by country and week. 
# In Stata, the solution would more likely involve -bysort-, which is a slightly cruder
# solution: we coerce the order of the rows and then just fill down over groups. 

# Here we make a small analytical data-set that I will first show. 
# Note that here we take the mean across weeks. This means that if 
# any week had an observation, there will be a number here. In reality,
# we have MVs in the df for the weeks after 2019 -- what we will do is use this 
# mean to fill forward. 
countryweek = df.groupby(["country", "week"]).mean()
grouped = countryweek[["driving", "transit", "deaths", "week_mean_death"]]
print(tabulate(grouped, tablefmt="pipe", headers="keys"))

/var/folders/r1/v24t28293vb4sxbzx6539ddw0000gn/T/ipykernel_1846/2470902517.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  countryweek = df.groupby(["country", "week"]).mean()


NameError: name 'tabulate' is not defined

In [ ]:
# Now, the idea is that we will forward fill down the missing weeks. 
df["week_mean_death"] = df.groupby(["country", "week"])["week_mean_death"].ffill()

In [ ]:
# As usual, let's check our work.
# Random sampling of groups is a little hard in Python unless the groups are all of the same size: 
# see discussion: https://stackoverflow.com/questions/22472213/python-random-selection-per-group

# But, we should probably know why we don't have equal sizes anyways!
# Here's some nice code: https://stackoverflow.com/questions/17995024/how-to-assign-a-name-to-the-size-column
quick = df[["country", "week", "deaths"]]
df["groupsize"] = quick.groupby(["country", "week"]).transform(np.size)
df.loc[df["groupsize"]<5]

In [ ]:
# Let's check for missing values while we're at it. 
df.isnull().sum()
df[df["week_mean_death"].isna()]

# So, we're running into the "53 weeks" problem. I think it's fine to just drop these three observations for
# simple exploratory analysis. 
df = df.loc[df["week"]<53]
df.isnull().sum()

# Let's finally check and see if that fixed things
groups = ["country", "week"]
print(f"The minimum number of country week pairings observered is {min(df.groupby(groups).size())}")

# Five seems much more reasonable

In [ ]:
# Now let's browse some samples -- this is important when you don't have a true
# spreadsheet of data. First, we make a groupby object...
gb = df.groupby(["country", "week"])

# Then we put the number of groups into a vector a
a=np.arange(gb.ngroups)

# Randomly shuffle its contents
np.random.shuffle(a)

# Then take a sample of five complete groups
sampleofgroups = df[gb.ngroup().isin(a[:5])]

# Omitting some variables for clarity
sampleofgroups = sampleofgroups[["country", "week", "year", "transit stations",\
                                 "transit", "deaths", "week_mean_death"]]

# And sorting the values to make the output readable
sampleofgroups.sort_values(by =["country", "week", "year"])

In [ ]:
# Finally, let's make a moving average of excess death.

# We first simply make a vector of deviations, which might be useful to inspect by hand
df["death_dev"] = df["deaths"] - df["week_mean_death"]

# Then we take this deviation as a percent of the mean for that week, historically
df["excess_mortality"] = (df["death_dev"]*100)/df["week_mean_death"]

# Then we turn it into a moving average
df["MA_EM"] = df["excess_mortality"].rolling(3, min_periods=2).mean()
df.dtypes

In [ ]:
# Let's now make some cool graphs. The first step is to see which countries have observations
# on the first set of variables. We can also drop observations from the period after which 
# information was generally no longer recorded publicly for Apple/Google. We also do this because
# much of the information for the two sets is the same, so we should focus on one set at once. 


countries1 = "Australia", "United States", "Chile", "Colombia", "United Kingdom", "New Zealand"
cutoff ='2022-04-01 00:00:00'
df_mod1 = df[(df["year"]>2019) & (df["date"]<cutoff) & df["country"].isin(countries1)]
df_mod1 = df_mod1[["date", "country", "driving", "transit", "walking", "excess_mortality"]]

# Let's also make this set one that takes moving averages for all variables...
keyvars = ["driving", "transit", "walking", "excess_mortality"]
df_mod1[keyvars] = \
    df_mod1.rolling(7)[keyvars].mean().fillna(method='bfill').fillna(method='ffill')

# Then, the next step is to melt our df after using only those countries with observations on these data.
# The idea here is that we want a dataset that, for Seaborn purposes, has values of the variables in 
# a separate "value column", with the variables themselves converted into values of a "variable-variable".
df_melt1 = pd.melt(df_mod1, ["country", 'date'])
df_melt1.head(20)

In [ ]:
plt.style.use('ggplot')
g1 = sns.FacetGrid(df_melt1, col='country', hue='variable', col_wrap=3,  palette="muted")
g1 = g1.map(sns.lineplot, 'date', 'value')
g1.add_legend()
g1.set_xticklabels(rotation=45)
g1.set_ylabels("Percent difference to baseline")
plt.show();